In [1]:
PATH_PREFIX = '../../'
import sys
sys.path.append(PATH_PREFIX)

In [2]:
import torch
import os
from utils.weight_sharing import *
from data.imagenette import *
from data.utils.download import *
from data.utils.imagenet_utils import *

BATCH_SIZE = 32

In [3]:
net = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

dat = ImagenetteDataset(BATCH_SIZE, os.path.join(PATH_PREFIX, 'data/imagenette/'), val_split=0.9)


Using cache found in /home/coupekv/.cache/torch/hub/pytorch_vision_v0.10.0


In [4]:
get_accuracy(net, dat.test_dl, 'cpu', topk=5)

96.1832046508789

In [5]:
ws_controller = WeightShare(net)
ws_controller.print_layers_info()

layer_name #weights #bias w_locked CR
features.0.0 864 0 False 1.00
features.1.conv.0.0 288 0 False 1.00
features.1.conv.1 512 0 False 1.00
features.2.conv.0.0 1536 0 False 1.00
features.2.conv.1.0 864 0 False 1.00
features.2.conv.2 2304 0 False 1.00
features.3.conv.0.0 3456 0 False 1.00
features.3.conv.1.0 1296 0 False 1.00
features.3.conv.2 3456 0 False 1.00
features.4.conv.0.0 3456 0 False 1.00
features.4.conv.1.0 1296 0 False 1.00
features.4.conv.2 4608 0 False 1.00
features.5.conv.0.0 6144 0 False 1.00
features.5.conv.1.0 1728 0 False 1.00
features.5.conv.2 6144 0 False 1.00
features.6.conv.0.0 6144 0 False 1.00
features.6.conv.1.0 1728 0 False 1.00
features.6.conv.2 6144 0 False 1.00
features.7.conv.0.0 6144 0 False 1.00
features.7.conv.1.0 1728 0 False 1.00
features.7.conv.2 12288 0 False 1.00
features.8.conv.0.0 24576 0 False 1.00
features.8.conv.1.0 3456 0 False 1.00
features.8.conv.2 24576 0 False 1.00
features.9.conv.0.0 24576 0 False 1.00
features.9.conv.1.0 3456 0 False 1.

In [6]:
len(ws_controller.model_layers)

53

In [ ]:
ws_controller.model_layers[2].plot_weight()

In [ ]:
classes = json.load(open('../data/imagenette/imagenet_class_index.json'))
classes_arr = []

for i in range(len(classes.keys())):
    classes_arr.append(classes[f'{i}'])

output = net(dat.test_dl.__getitem__(1)[0].unsqueeze(0))

probabilities = torch.nn.functional.softmax(output[0], dim=0)

top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(classes_arr[top5_catid[i]], top5_prob[i].item(), top5_catid[i])